In [31]:
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

**8번**

In [4]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.uint8)

In [10]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=11)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=11)

In [15]:
RandomForest_clf = RandomForestClassifier(n_estimators=100, max_depth = 5, random_state = 11)
ExtraTrees_clf = ExtraTreesClassifier(n_estimators=100, random_state=11)
SVM_clf = LinearSVC(max_iter=1000, random_state=11)
MLP_clf = MLPClassifier(random_state=11)

In [16]:
estimators = [RandomForest_clf, ExtraTrees_clf, SVM_clf, MLP_clf]
for estimator in estimators :
  estimator.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [17]:
for estimator in estimators :
  print(estimator.score(X_val, y_val))

0.8565
0.9695
0.8741
0.9598


In [18]:
Estimators = [
    ("RandomForest_clf", RandomForest_clf),
    ("ExtraTrees_clf", ExtraTrees_clf),
    ("SVM_clf", SVM_clf),
    ("MLP_clf", MLP_clf),
]

In [19]:
Voting_clf = VotingClassifier(Estimators)
Voting_clf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


VotingClassifier(estimators=[('RandomForest_clf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=5,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.

In [21]:
for estimator in Voting_clf.estimators_ :
  print(estimator.score(X_val, y_val))

0.8565
0.9695
0.8741
0.9598


In [24]:
del Voting_clf.estimators_[0,2]
Voting_clf.voting = "soft"
Voting_clf.score(X_test, y_test)

0.9631

In [25]:
Voting_clf.voting = "hard"
Voting_clf.score(X_test, y_test)

0.9627

**9번**

In [26]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [27]:
X_val_predictions

array([[9., 9., 9., 9.],
       [1., 1., 1., 1.],
       [9., 9., 9., 9.],
       ...,
       [5., 5., 5., 5.],
       [9., 9., 5., 9.],
       [9., 9., 9., 9.]], dtype=float32)

In [37]:
blender = RandomForestClassifier(n_estimators=200, random_state=11)
blender.fit(X_val_predictions, y_val)
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

In [38]:
for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)
y_pred = blender.predict(X_test_predictions)

In [39]:
accuracy_score(y_test, y_pred)

0.9672